# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import string 
import sys 

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\loisn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\loisn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\loisn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DS_messages', engine)
X = df['message']
y = df[df.columns[4:]]
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    words = [w for w in tokens if w not in stop_words]
    
    # Reduce words to their stems
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(w) for w in words]
    
    # Reduce words to their root form
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w) for w in stemmed]
    
    return lemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# set up pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000280B2C84310>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Predict on test set 
y_test_pred = pipeline.predict(X_test)
categories = y.columns.tolist()

# Test model on test set
print(classification_report(y_test, y_test_pred, target_names=categories, zero_division=0))


                        precision    recall  f1-score   support

               related       0.86      0.93      0.89      6554
               request       0.82      0.52      0.63      1493
                 offer       0.00      0.00      0.00        41
           aid_related       0.76      0.71      0.74      3619
          medical_help       0.66      0.07      0.13       715
      medical_products       0.84      0.06      0.12       436
     search_and_rescue       1.00      0.01      0.02       234
              security       0.50      0.01      0.01       161
              military       0.67      0.05      0.09       278
                 water       0.85      0.33      0.48       528
                  food       0.87      0.49      0.63       958
               shelter       0.88      0.25      0.39       806
              clothing       0.77      0.07      0.14       134
                 money       0.78      0.03      0.06       218
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
import joblib 

parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__criterion': ['entropy'],
    'clf__estimator__max_features': ['log2']
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3, verbose=3)

In [12]:
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50, score=0.237, total= 1.7min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50, score=0.244, total= 1.8min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.5min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=50, score=0.238, total= 1.8min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100, score=0.233, total= 3.3min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100, score=0.249, total= 3.5min
[CV] clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__max_features=log2, clf__estimator__n_estimators=100, score=0.236, total= 3.6min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 15.7min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000280B2C84310>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced')))]),
             param_grid={'clf__estimator__criterion': ['entropy'],
                         'clf__estimator__max_features': ['log2'],
                         'clf__estimator__n_estimators': [50, 100]},
             verbose=3)

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import string 
import sys 

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB


In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DS_messages', engine)
X = df['message']
y = df[df.columns[4:]]
categories = y.columns.tolist()

In [ ]:
def tokenize(text):
    # normalize text and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    stop_words = stopwords.words("english")
    words = [w for w in tokens if w not in stop_words]
    
    # Reduce words to their stems
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(w) for w in words]
    
    # Reduce words to their root form
    lemmatizer = WordNetLemmatizer()
    lemmed = [lemmatizer.lemmatize(w) for w in stemmed]
    
    return lemmed

In [ ]:
def build_model():
    """Returns the GridSearchCV model
    Args:
        None
    Returns:
        cv: Grid search model object
    """
    clf = RandomForestClassifier(n_estimators=100)

    # The pipeline has tfidf, dimensionality reduction, and classifier
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(BernoulliNB()))
    ])

    # Parameters for GridSearchCV
    param_grid = {
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__ngram_range': ((1, 1), (1,2)),
        'vect__max_features': (None, 5000,10000),
        'tfidf__use_idf': (True, False)
    }

    cv = GridSearchCV(pipeline, param_grid)

    return cv

In [ ]:
# Create grid search object
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1,1), (1,2)),
    'vect__max_features': (None, 5000,10000),
    'clf__estimator__n_estimators':(50, 100), 
    'clf__estimator__min_samples_split':(2, 5),
    'tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid = parameters)


In [ ]:
# Train tuned model
cv.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
# Predict tuned model on test set 
y_test_pred = cv.predict(X_test)
categories = y.columns.tolist()

# Evaluate tuned model on test set
print(classification_report(y_test, y_test_pred, target_names=categories, zero_division=0))


                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      6554
               request       0.85      0.40      0.55      1493
                 offer       0.00      0.00      0.00        41
           aid_related       0.81      0.56      0.66      3619
          medical_help       0.83      0.01      0.03       715
      medical_products       0.64      0.02      0.04       436
     search_and_rescue       0.00      0.00      0.00       234
              security       0.50      0.01      0.01       161
              military       1.00      0.00      0.01       278
                 water       0.90      0.12      0.20       528
                  food       0.92      0.18      0.31       958
               shelter       0.94      0.08      0.15       806
              clothing       0.70      0.05      0.10       134
                 money       0.75      0.03      0.05       218
        missing_people       0.00      

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.